### Задание

Создайте систему компьютерного зрения, которая будет определять тип геометрической фигуры. Используя подготовленную базу и шаблон ноутбука проведите серию экспериментов по перебору гиперпараметров нейронной сети, распознающей три категории изображений (треугольник, круг, квадрат).

1. Поменяйте количество нейронов в сети, используя следующие значения:

- один слой 10 нейронов
- один слой 100 нейронов
- один слой 5000 нейронов.

2. Поменяйте активационную функцию в скрытых слоях с `relu` на `linear`.
3. Поменяйте размеры batch_size:
- 10
- 100
- 1000

4. Выведите на экран получившиеся точности.

Всего должно получиться 18 комбинаций указанных параметров.

Создайте сравнительную таблицу по результатам проведенных тестов.

In [ ]:
# Подключение класса для создания нейронной сети прямого распространения
from tensorflow.keras.models import Sequential
# Подключение класса для создания полносвязного слоя
from tensorflow.keras.layers import Dense
# Подключение оптимизатора
from tensorflow.keras.optimizers import Adam
# Подключение утилит для to_categorical
from tensorflow.keras import utils
# Подключение библиотеки для загрузки изображений
from tensorflow.keras.preprocessing import image
# Подключение библиотеки для работы с массивами
import numpy as np
# Подключение библиотек для отрисовки изображений
import matplotlib.pyplot as plt
# Подключение модуля для работы с файлами
import os
# Вывод изображения в ноутбуке, а не в консоли или файле
%matplotlib inline

In [ ]:
# Загрузка датасета из облака
import gdown
gdown.download('https://storage.yandexcloud.net/aiueducation/Content/base/l3/hw_light.zip', None, quiet=True)

'hw_light.zip'

In [ ]:
# Распаковываем архив hw_light.zip в папку hw_light
!unzip -q hw_light.zip

replace hw_light/0/1.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace hw_light/0/10.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace hw_light/0/100.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace hw_light/0/101.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace hw_light/0/102.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace hw_light/0/11.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace hw_light/0/12.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace hw_light/0/13.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace hw_light/0/14.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace hw_light/0/15.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace hw_light/0/16.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace hw_light/0/17.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace hw_light/0/18.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace hw_light/0/19.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 
error:  invalid response [{ENTER}]
replace hw_light/0/19.jpg?

In [ ]:
# Загрузка и подготовка данных
data_dir = "./hw_light"

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    data_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical'
)

# Гиперпараметры для экспериментов
neurons_options = [10, 100, 5000]
activation_options = ['relu', 'linear']
batch_sizes = [10, 100, 1000]

results = []

# Перебор всех комбинаций
for neurons in neurons_options:
    for activation in activation_options:
        for batch_size in batch_sizes:

            # Создание модели
            model = Sequential([
                Flatten(input_shape=(64, 64, 3)),
                Dense(neurons, activation=activation),
                Dense(3, activation='softmax')
            ])

            model.compile(optimizer=Adam(),
                          loss='categorical_crossentropy',
                          metrics=['accuracy'])

            # Обучение модели
            history = model.fit(
                train_generator,
                epochs=5,
                batch_size=batch_size,
                validation_data=test_generator,
                verbose=0
            )

            # Оценка модели
            _, test_acc = model.evaluate(test_generator, verbose=0)

            # Сохранение результатов
            results.append({
                'Neurons': neurons,
                'Activation': activation,
                'Batch Size': batch_size,
                'Accuracy': test_acc
            })

            print(f"Neurons: {neurons}, Activation: {activation}, Batch Size: {batch_size}, Accuracy: {test_acc:.4f}")

# Создание таблицы результатов
results_df = pd.DataFrame(results)
print(results_df)


Found 302 images belonging to 3 classes.
Found 302 images belonging to 3 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Neurons: 10, Activation: relu, Batch Size: 10, Accuracy: 0.5464
Neurons: 10, Activation: relu, Batch Size: 100, Accuracy: 0.3377
Neurons: 10, Activation: relu, Batch Size: 1000, Accuracy: 0.6358
Neurons: 10, Activation: linear, Batch Size: 10, Accuracy: 0.5166
Neurons: 10, Activation: linear, Batch Size: 100, Accuracy: 0.8377
Neurons: 10, Activation: linear, Batch Size: 1000, Accuracy: 0.7914
Neurons: 100, Activation: relu, Batch Size: 10, Accuracy: 0.8245
Neurons: 100, Activation: relu, Batch Size: 100, Accuracy: 0.7748
Neurons: 100, Activation: relu, Batch Size: 1000, Accuracy: 0.7682
Neurons: 100, Activation: linear, Batch Size: 10, Accuracy: 0.6821
Neurons: 100, Activation: linear, Batch Size: 100, Accuracy: 0.8113
Neurons: 100, Activation: linear, Batch Size: 1000, Accuracy: 0.6258
Neurons: 5000, Activation: relu, Batch Size: 10, Accuracy: 0.7152
Neurons: 5000, Activation: relu, Batch Size: 100, Accuracy: 0.6921
Neurons: 5000, Activation: relu, Batch Size: 1000, Accuracy: 0.7781
N